<a href="https://colab.research.google.com/github/SnapRock/desafio_imersao_ia_alura_receita_facil/blob/main/desafio_imersao_ia_alura_receita_facil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
import numpy as np
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Modelo de embedding
model = "models/embedding-001"

In [ ]:
def gerar_embedding(texto, titulo):
    return genai.embed_content(
        model=model,
        content=texto,
        task_type="RETRIEVAL_DOCUMENT",
        title=titulo
    )["embedding"]

In [ ]:
def gerar_receita(titulo_receita):
    """Gera uma receita formal a partir do título."""
    prompt = f"""
    Você é um chef experiente e criativo.
    Crie uma receita formal e completa, com ingredientes e instruções detalhadas, para um prato chamado '{titulo_receita}'.

    A receita deve incluir:
    - Título da Receita
    - Ingredientes: Listados em formato de marcadores, com quantidades e unidades de medida.
    - Instruções: Numeradas sequencialmente, descrevendo cada passo da preparação de forma clara e concisa.

    Seja criativo e invente uma receita deliciosa e plausível!
    """

    model_2 = genai.GenerativeModel("gemini-1.0-pro")
    response = model_2.generate_content(prompt)
    return response.text

In [ ]:
def gerar_embedding_query(texto):
    """Gera o embedding de um texto usando o modelo especificado."""
    return genai.embed_content(
        model="models/embedding-001",
        content=texto,
        task_type="RETRIEVAL_QUERY"
    )["embedding"]

In [ ]:
def extrair_ingredientes(receita, titulo_receita):
    """Extrai os ingredientes da receita usando embeddings."""
    embedding_receita = gerar_embedding(receita, titulo=titulo_receita)

    ingredientes = []
    for linha in receita.split('\n'):
        linha = linha.strip()
        if linha: # Ignora linhas vazias
            # Usamos o título "Ingrediente da Receita" para cada linha
            embedding_linha = gerar_embedding(linha, titulo="Ingrediente da Receita")
            similaridade = np.dot(embedding_receita, embedding_linha)
            # Ajuste o limite de similaridade conforme necessário
            if similaridade > 0.8:
                ingredientes.append(linha)
    return ingredientes

In [ ]:
def gerar_lista_compras(ingredientes):
    """Gera a lista de compras a partir dos ingredientes."""
    lista_compras = []
    for ingrediente in ingredientes:
        # Divida a linha do ingrediente em quantidade, unidade e nome
        partes = ingrediente.split(' ')
        quantidade = partes[0]
        unidade = partes[1] if len(partes) > 2 else ""
        nome = ' '.join(partes[2:]) if len(partes) > 2 else partes[1]

        # Adiciona o ingrediente à lista de compras
        lista_compras.append(f"{quantidade} {unidade} {nome}")

    return lista_compras

In [ ]:
def formatar_lista_compras(lista_compras):
    """Formata a lista de compras em um formato comercialmente viável."""
    lista_compras_str = '\n'.join(lista_compras)
    return gerar_embedding_query(lista_compras_str)

In [ ]:
def interpretar_lista_compras(lista_compras):
    """Interpreta a lista de compras usando um modelo de linguagem
       para sugerir produtos específicos e formatação comercial.
    """
    prompt = f"""
    Você é um assistente de compras experiente.
    Com base na seguinte lista de compras, sugira produtos específicos
    e formate a lista de forma clara e organizada para um supermercado:

    {lista_compras}

    Considere marcas populares, tamanhos de embalagem comuns e
    especificações relevantes para cada item.
    """
    model_2 = genai.GenerativeModel("gemini-1.0-pro")
    response = model_2.generate_content(prompt)
    return response.text

In [ ]:
if __name__ == "__main__":
    titulo_receita = input("Qual receita você gostaria de criar hoje? ")
    receita = gerar_receita(titulo_receita)
    print(receita)

    #Ingredientes Extraídos
    ingredientes = extrair_ingredientes(receita, titulo_receita) # Passamos o título da receita

    #Lista de Compras Bruta
    lista_compras = gerar_lista_compras(ingredientes)

    #Lista de Compras Interpretada
    lista_compras_interpretada = interpretar_lista_compras(lista_compras)
    print("\nLista de Compras:")
    print(lista_compras_interpretada)
